In [1]:
#TOPSIS-AHP
import numpy as np               # for linear algebra
import pandas as pd              # for tabular output
from scipy.stats import rankdata # for ranking the candidates
import matplotlib.pyplot as plt

In [2]:
# The given data encoded into vectors and matrices

attributes = np.array(["Security", "QOS", "Cost", "Reputation", "Financial", "Customer", "Performance"])
candidates = np.array(["Amazon","Microsoft","Google","Alibaba","Oracle","IBM"])
raw_data = np.array([
    [4.6, 4.0, 0.120, 4.5, 4.5, 4.5, 4.0],
    [4.5, 4.0, 0.107, 4.4, 4.0, 4.3, 4.0],
    [4.6, 4.0, 0.067, 4.5, 4.0, 4.5, 4.0],
    [4.3, 5.0, 0.198, 4.3, 5.0, 4.4, 4.0],
    [4.4, 5.0, 0.119, 4.3, 3.0, 4.4, 4.0],
    [4.4, 5.0, 0.106, 4.3, 1.0, 4.2, 4.0],
])

weights  = np.array([0.386, 0.077, 0.193, 0.055, 0.129, 0.096, 0.064])



benefit_attributes = set([0, 1, 0, 3, 4, 5, 6])

# Display the raw data we have
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
Amazon,4.6,4.0,0.120,4.5,4.5,4.5,4.0
Microsoft,4.5,4.0,0.107,4.4,4.0,4.3,4.0
Google,4.6,4.0,0.067,4.5,4.0,4.5,4.0
Alibaba,4.3,5.0,0.198,4.3,5.0,4.4,4.0
Oracle,4.4,5.0,0.119,4.3,3.0,4.4,4.0
IBM,4.4,5.0,0.106,4.3,1.0,4.2,4.0


In [3]:
m = len(raw_data)
n = len(attributes)
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.sqrt(column @ column)

raw_data /= divisors

columns = ["Security", "QOS", "Cost", "Reputation", "Financial", "Customer", "Performance"]
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
Amazon,0.420306,0.360668,0.389456,0.419026,0.481759,0.418990,0.408248
Microsoft,0.411169,0.360668,0.347265,0.409715,0.428230,0.400368,0.408248
Google,0.420306,0.360668,0.217447,0.419026,0.428230,0.418990,0.408248
Alibaba,0.392895,0.450835,0.642603,0.400403,0.535288,0.409679,0.408248
Oracle,0.402032,0.450835,0.386211,0.400403,0.321173,0.409679,0.408248
IBM,0.402032,0.450835,0.344020,0.400403,0.107058,0.391057,0.408248


In [4]:
raw_data *= weights
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
Amazon,0.162238,0.027771,0.075165,0.023046,0.062147,0.040223,0.026128
Microsoft,0.158711,0.027771,0.067022,0.022534,0.055242,0.038435,0.026128
Google,0.162238,0.027771,0.041967,0.023046,0.055242,0.040223,0.026128
Alibaba,0.151657,0.034714,0.124022,0.022022,0.069052,0.039329,0.026128
Oracle,0.155184,0.034714,0.074539,0.022022,0.041431,0.039329,0.026128
IBM,0.155184,0.034714,0.066396,0.022022,0.013810,0.037542,0.026128


In [5]:
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = raw_data[:,j]
    max_val = np.max(column)
    min_val = np.min(column)
    
    # See if we want to maximize benefit or minimize cost (for PIS)
    if j in benefit_attributes:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

pd.DataFrame(data=[a_pos, a_neg], index=["$Sj^+$", "$Sj^-$"], columns=columns)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
$Sj^+$,0.162238,0.034714,0.041967,0.023046,0.069052,0.040223,0.026128
$Sj^-$,0.151657,0.027771,0.124022,0.022022,0.013810,0.037542,0.026128


In [6]:
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = raw_data[i] - a_pos
    diff_neg = raw_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

pd.DataFrame(data=zip(sp, sn, cs), index=candidates, columns=["$di^+$", "$di^-$", "$Ci$"])


,$di^+$,$di^-$,$Ci$
Amazon,0.034612,0.069596,0.667858
Microsoft,0.029708,0.070826,0.704497
Google,0.015457,0.092573,0.856917
Alibaba,0.082746,0.055705,0.402345
Oracle,0.043306,0.057231,0.569252
IBM,0.060880,0.058150,0.488533
